In [1]:
import rootutils
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.spatial.transform import Rotation as R
import cv2

In [2]:
PROJECT_ROOT_DIR = rootutils.find_root(search_from=os.path.abspath(''), indicator=".project-root")
PROJECT_DATA_DIR = str(PROJECT_ROOT_DIR) + '/data/'

In [3]:
path = PROJECT_DATA_DIR + 'raw'

obj = os.scandir(path=path)

result_entry = []

# List all files and directories in the specified path
print("Files and Directories in '% s':" % path)
for entry in obj:
    if entry.is_dir() or entry.is_file() and entry.name.endswith('.yml'):
        result_entry.append(entry.path)

Files and Directories in '/home/ghost/Documents/WorkSpace/&HXD/Trainning/VisualPropModel/data/raw':


## Generate Trainning Data
#### Input: 3 Markers' Poses
#### Output: Contact Position

In [4]:
trainingData = np.array([])
for number in range(len(result_entry)):
    file_str = result_entry[number]
    # print(file_str)   
    f = cv2.FileStorage(file_str,cv2.FILE_STORAGE_READ)
    num_data = int(f.getNode("number_of_points").real())
    gt_point = np.array([])
    bc_top = np.array([])
    bc_bot1 = np.array([])
    bc_bot2 = np.array([])

    a = f.getNode("points")
    for index in range(num_data):
        gt = np.array([a.at(index).getNode('gt_point').at(0).real(),
                       a.at(index).getNode('gt_point').at(1).real(),
                       a.at(index).getNode('gt_point').at(2).real()])
        
        eulerAng1 = R.from_quat([a.at(index).getNode('bc_top').at(3).real(),
                                a.at(index).getNode('bc_top').at(4).real(),
                                a.at(index).getNode('bc_top').at(5).real(),
                                a.at(index).getNode('bc_top').at(6).real()]).as_euler('xyz', degrees=True)

        eulerAng2 = R.from_quat([a.at(index).getNode('bc_bot1').at(3).real(),
                                 a.at(index).getNode('bc_bot1').at(4).real(),
                                 a.at(index).getNode('bc_bot1').at(5).real(),
                                 a.at(index).getNode('bc_bot1').at(6).real()]).as_euler('xyz', degrees=True)
        
        eulerAng3 = R.from_quat([a.at(index).getNode('bc_bot2').at(3).real(),
                                 a.at(index).getNode('bc_bot2').at(4).real(),
                                 a.at(index).getNode('bc_bot2').at(5).real(),
                                 a.at(index).getNode('bc_bot2').at(6).real()]).as_euler('xyz', degrees=True)

        top = np.array([a.at(index).getNode('bc_top').at(0).real(),
                        a.at(index).getNode('bc_top').at(1).real(),
                        a.at(index).getNode('bc_top').at(2).real(),
                        eulerAng1[0],
                        eulerAng1[1],
                        eulerAng1[2]])

        bot1 = np.array([a.at(index).getNode('bc_bot1').at(0).real(),
                         a.at(index).getNode('bc_bot1').at(1).real(),
                         a.at(index).getNode('bc_bot1').at(2).real(),
                         eulerAng2[0],
                         eulerAng2[1],
                         eulerAng2[2]])
                    
        bot2 = np.array([a.at(index).getNode('bc_bot2').at(0).real(),
                         a.at(index).getNode('bc_bot2').at(1).real(),
                         a.at(index).getNode('bc_bot2').at(2).real(),
                         eulerAng3[0],
                         eulerAng3[1],
                         eulerAng3[2]])
        if index == 0:
            gt_point = gt.reshape(1,3)
            bc_top = top.reshape(1,6)
            bc_bot1 = bot1.reshape(1,6)
            bc_bot2 = bot2.reshape(1,6)
            
        else:
            gt_point = np.concatenate((gt_point,gt.reshape(1,3)),axis=0)
            bc_top = np.concatenate((bc_top,top.reshape(1,6)),axis=0)
            bc_bot1 = np.concatenate((bc_bot1,bot1.reshape(1,6)),axis=0)
            bc_bot2 = np.concatenate((bc_bot2,bot2.reshape(1,6)),axis=0)

    gt_point = np.array(gt_point)
    bc_top = np.array(bc_top)
    bc_bot1 = np.array(bc_bot1)
    bc_bot2 = np.array(bc_bot2)

    # print(gt_point.shape)

    stop = 300

    sub_gt_point = gt_point[0:stop]
    sub_bc_top = bc_top[0:stop]
    sub_bc_bot1 = bc_bot1[0:stop]
    sub_bc_bot2= bc_bot2[0:stop]

    data = np.concatenate((sub_bc_top,sub_bc_bot1,sub_bc_bot2, sub_gt_point),axis=1)
    if number == 0:
        trainingData = data
    else:
        trainingData = np.concatenate((trainingData,data),axis=0)

np.save(PROJECT_DATA_DIR + 'bc_3_trainingData.npy',trainingData)

## Generate Trainning Data
#### Input: 2 Markers' Poses
#### Output: Contact Position

In [5]:
trainingData = np.array([])
for number in range(len(result_entry)):
    file_str = result_entry[number]
    f = cv2.FileStorage(file_str,cv2.FILE_STORAGE_READ)
    num_data = int(f.getNode("number_of_points").real())
    gt_point = np.array([])
    bc_bot1 = np.array([])
    bc_bot2 = np.array([])

    a = f.getNode("points")
    for index in range(num_data):
        gt = np.array([a.at(index).getNode('gt_point').at(0).real(),
                       a.at(index).getNode('gt_point').at(1).real(),
                       a.at(index).getNode('gt_point').at(2).real()])
        
        eulerAng2 = R.from_quat([a.at(index).getNode('bc_bot1').at(3).real(),
                                 a.at(index).getNode('bc_bot1').at(4).real(),
                                 a.at(index).getNode('bc_bot1').at(5).real(),
                                 a.at(index).getNode('bc_bot1').at(6).real()]).as_euler('xyz', degrees=True)
        
        eulerAng3 = R.from_quat([a.at(index).getNode('bc_bot2').at(3).real(),
                                 a.at(index).getNode('bc_bot2').at(4).real(),
                                 a.at(index).getNode('bc_bot2').at(5).real(),
                                 a.at(index).getNode('bc_bot2').at(6).real()]).as_euler('xyz', degrees=True)

        bot1 = np.array([a.at(index).getNode('bc_bot1').at(0).real(),
                         a.at(index).getNode('bc_bot1').at(1).real(),
                         a.at(index).getNode('bc_bot1').at(2).real(),
                         eulerAng2[0],
                         eulerAng2[1],
                         eulerAng2[2]])
                    
        bot2 = np.array([a.at(index).getNode('bc_bot2').at(0).real(),
                         a.at(index).getNode('bc_bot2').at(1).real(),
                         a.at(index).getNode('bc_bot2').at(2).real(),
                         eulerAng3[0],
                         eulerAng3[1],
                         eulerAng3[2]])
        if index == 0:
            gt_point = gt.reshape(1,3)
            bc_bot1 = bot1.reshape(1,6)
            bc_bot2 = bot2.reshape(1,6)
            
        else:
            gt_point = np.concatenate((gt_point,gt.reshape(1,3)),axis=0)
            bc_bot1 = np.concatenate((bc_bot1,bot1.reshape(1,6)),axis=0)
            bc_bot2 = np.concatenate((bc_bot2,bot2.reshape(1,6)),axis=0)

    gt_point = np.array(gt_point)
    bc_bot1 = np.array(bc_bot1)
    bc_bot2 = np.array(bc_bot2)


    stop = 300

    sub_gt_point = gt_point[0:stop]
    sub_bc_bot1 = bc_bot1[0:stop]
    sub_bc_bot2= bc_bot2[0:stop]

    data = np.concatenate((sub_bc_bot1,sub_bc_bot2, sub_gt_point),axis=1)
    if number == 0:
        trainingData = data
    else:
        trainingData = np.concatenate((trainingData,data),axis=0)

np.save(PROJECT_DATA_DIR + 'bc_2_trainingData.npy',trainingData)

In [6]:
data_path: str = PROJECT_DATA_DIR + 'bc_3_trainingData.npy'
data = np.load(data_path)


In [8]:
data.shape

(40477, 21)

In [9]:
data_input = data[0,:]

In [10]:
print(data[0,:])

[ 1.71083605e+00  3.67891520e-01  1.52066350e-02  2.34989235e+00
  6.35511935e-01  9.99698637e-01  1.71083605e+00  3.67891520e-01
  1.52066350e-02  2.34989235e+00  6.35511935e-01  9.99698637e-01
  1.71083605e+00  3.67891520e-01  1.52066350e-02  2.34989235e+00
  6.35511935e-01  9.99698637e-01  1.30348873e+01  7.16708755e+01
 -1.12086630e+01]


In [24]:
print(data[0,0 : -3])

[1.71083605 0.36789152 0.01520663 2.34989235 0.63551193 0.99969864
 1.71083605 0.36789152 0.01520663 2.34989235 0.63551193 0.99969864
 1.71083605 0.36789152 0.01520663 2.34989235 0.63551193 0.99969864]


In [27]:
print(data[0, -3 : len(data[0,])])

[ 13.03488731  71.67087555 -11.20866299]
